<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module5/M5_1/1_Network_TransportingCoal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Problems: Transporting Coal
**OPIM 5641: Business Decision Modeling - University of Connecticut**

--------------------------------------------
Make sure you are consistent with your demand/supply pairs!

This example has a little bit of a different flavor - now we have to consider shipping costs AND production costs in the objective function...

* Please refer to Powell Chapter 10 for more details and examples.
* Pyomo Cookbook (related example): https://github.com/jckantor/ND-Pyomo-Cookbook/blob/master/notebooks/03.01-Transportation-Networks.ipynb

-------------------------------------------------------------------------

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
    
from pyomo.environ import *

# Example
1. Transporting Coal. The Calcio Coal Company produces
coal at four mines and ships it to four power plants (P1-P4).
The cost per ton of producing coal and the production
capacity (in tons) for each mine are known. The number
of tons of coal demanded by each customer is also known.
The cost (in dollars) of shipping a ton of coal from a mine to
each plant is available as well. The following table provides
the data:

Name |P1| P2| P3| P4| Capacity| Cost
---|---|---|---|---|---|---
Mine 1| 9| 15| 8| 10| 125| 50
Mine 2| 7| 15| 14| 12| 100| 57
Mine 3| 5| 5| 11| 12| 150| 55
Mine 4| 3| 6| 8| 11| 120| 61
Demands| 110| 115| 135| 130

1. Calcio wishes to minimize the cost of transporting coal from
its mines to its plants. What is the minimum cost?
2. What is the transportation schedule that achieves the minimum cost in (a)?

In [ ]:
# first, let's make sure we have enough supply relative to the demand
Supply = 125 + 100 + 150 + 120
Demand = 110 + 115 + 135 + 130
Supply-Demand # yep! so our network won't freak out.

5

# Thoughts/Problem Set-Up
So we have four mines that need to feed four power plants. A classic network problem subject to supply/capacity and demand constraints.

## Objective Function
Let $M_{11}$ be the arc from Mine 1 to Power Plant 1, Let $M_{12}$ be the arc from Mine 1 to Power Plant 2 ... 

We want minimize the cost of the shipping routes (arcs) from the mines from the power plants.

$\min(\text{Shipping Cost}) =  \\
\quad\quad\quad\quad\quad\quad 9*M_{11} + 15*M_{12} + 8*M_{13} + 10*M_{14} + \\
\quad\quad\quad\quad\quad\quad 7*M_{21} + 15*M_{22} + 14*M_{23} + 12*M_{24} + \\
\quad\quad\quad\quad\quad\quad 5*M_{31} + 5*M_{32} + 11*M_{33} + 12*M_{34} + \\
\quad\quad\quad\quad\quad\quad 3*M_{41} + 6*M_{42} + 8*M_{43} + 11*M_{44}$

But those are only the cost of the routes! We also need to cost corresponding to the production cost at each mine. 

So we update the objective function to account for the other info.

$\min(\text{Shipping and Production Cost}) = \\
\quad\quad\quad\quad\quad\quad 9*M_{11} + 15*M_{12} + 8*M_{13} + 10*M_{14}) + \\
\quad\quad\quad\quad\quad\quad                    7*M_{21} + 15*M_{22} + 14*M_{23} + 12*M_{24} + \\
 \quad\quad\quad\quad\quad\quad                   5*M_{31} + 5*M_{32} + 11*M_{33} + 12*M_{34} + \\
\quad\quad\quad\quad\quad\quad                    3*M_{41} + 6*M_{42} + 8*M_{43} + 11*M_{44} + \\
\quad\quad\quad\quad\quad\quad                    50*M_{11} + 50*M_{12} + 50*M_{13} + 50*M_{14} + \\
\quad\quad\quad\quad\quad\quad                    57*M_{21} + 57*M_{22} + 55*M_{23} + 61*M_{24} + \\
\quad\quad\quad\quad\quad\quad                    55*M_{31} + 55*M_{32} + 55*M_{33} + 55*M_{34} + \\
\quad\quad\quad\quad\quad\quad                    61*M_{41} + 61*M_{42} + 61*M_{43} + 61*M_{44}$


**WOOF!** Long equation but it is what it is... que sera, sera... later on you will see this is similar to a trans-shipment problem... you just keep adding costs to your objective function and try to minimize it.

## Constraints
Now remember, you have both supply/capacity and demand constraints.


### Supply Constraints

$M_{11} + M_{12} + M_{13} + M_{14} \leq 125$

In words, the total amount shipped out of Mine 1 ($M1$) must be less than or equal to the Mine 1 capacity. For the other mines (2, 3, 4), we have similar constraints.

$M_{21} + M_{22} + M_{23} + M_{24} \leq 100$

$M_{31} + M_{32} + M_{33} + M_{34} \leq 150$

$M_{41} + M_{42} + M_{43} + M_{44} \leq 120$

### Demand Constraints
Demand constraints are at the power plants.

$M_{11} + M_{21} + M_{31} + M_{41} \geq 110$ 

In words, the amount received at Power Plant 1 ($P1$) must be greater than or equal to the $P1$ requirement. Similarly, for the other Power Plants, the demand constraint becomes:

$M_{11} + M_{22} + M_{32} + M_{12} \geq 115$ 

$M_{13} + M_{23} + M_{33} + M_{13} \geq 135$ 

$M_{14} + M_{24} + M_{34} + M_{14} \geq 130$ 



# Implementation in Pyomo
We will use the cookbook example because it would be a monster to type all of the constraints - and we would probably make a mistake!

In [ ]:
Demand = {
   'P1':   110,        # P1
   'P2':   115,        # P2
   'P3':   135,        # P3
   'P4':   130,        # P4
}

Supply = {
   'M1':   125,      # M1
   'M2':   100,      # M2
   'M3':   150,      # M3
   'M4':   120       # M4
}

# remember, it is DEMAND - SUPPLY pairs (might seem backwards)
# T = transportation routes/costs...
T = {
    ('P1','M1'): 9,
    ('P2','M1'): 15,
    ('P3','M1'): 8,
    ('P4','M1'): 10,
    ('P1','M2'): 7,
    ('P2','M2'): 15,
    ('P3','M2'): 14,
    ('P4','M2'): 12,
    ('P1','M3'): 5,
    ('P2','M3'): 5,
    ('P3','M3'): 11,
    ('P4','M3'): 12,
    ('P1','M4'): 3,
    ('P2','M4'): 6,
    ('P3','M4'): 8,
    ('P4','M4'): 11
}

# P for production costs
P = {
    ('P1','M1'): 50,
    ('P2','M1'): 50,
    ('P3','M1'): 50,
    ('P4','M1'): 50,
    ('P1','M2'): 57,
    ('P2','M2'): 57,
    ('P3','M2'): 57,
    ('P4','M2'): 57,
    ('P1','M3'): 55,
    ('P2','M3'): 55,
    ('P3','M3'): 55,
    ('P4','M3'): 55,
    ('P1','M4'): 61,
    ('P2','M4'): 61,
    ('P3','M4'): 61,
    ('P4','M4'): 61
}

Now that this is all defined, you can plug and chug (copy!) the code from before. Be efficient!

In [ ]:
# define the model
# Step 0: Create an instance of the model
model = ConcreteModel()

# Step 1: Define index sets
CUS = list(Demand.keys()) # CUS is for customers (demand)
SRC = list(Supply.keys()) # SRC is for source (supply/capcity)

# Step 2: Define the decision 
model.x = Var(CUS, SRC, domain = NonNegativeReals)

Remember, this is just your shipping cost... you also need your production costs!

In [ ]:
# Step 3: Define Objective
model.Cost = Objective(
    expr = sum([T[c,s]*model.x[c,s] for c in CUS for s in SRC] + # shipping costs
               [P[c,s]*model.x[c,s] for c in CUS for s in SRC]), # production costs
    sense = minimize)

# notice how similar the expressions and data setup are...

Add constraints!

In [ ]:
# Step 4: Add Constraints (note the ease in which this happens with for loops!)
model.src = ConstraintList()
for s in SRC:
    model.src.add(sum([model.x[c,s] for c in CUS]) <= Supply[s])
        
model.dmd = ConstraintList()
for c in CUS:
    model.dmd.add(sum([model.x[c,s] for s in SRC]) >= Demand[c])

Let's see what this model looks like...

In [ ]:
model.pprint()

5 Set Declarations
    dmd_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    src_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   16 : {('P1', 'M1'), ('P1', 'M2'), ('P1', 'M3'), ('P1', 'M4'), ('P2', 'M1'), ('P2', 'M2'), ('P2', 'M3'), ('P2', 'M4'), ('P3', 'M1'), ('P3', 'M2'), ('P3', 'M3'), ('P3', 'M4'), ('P4', 'M1'), ('P4', 'M2'), ('P4', 'M3'), ('P4', 'M4')}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'P1', 'P2', 'P3', 'P4'}
    x_index_1 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :   

Solve it!

In [ ]:
# solve the model and show results
results = SolverFactory('cbc').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 30770.0
  Upper bound: 30770.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 17
  Number of nonzeros: 16
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of creat

# Results
Even if you're not as comfortable with the syntax to start, you have to know how to apply it first - then try to tease out what each line means! 

In [ ]:
# print the cost - checks out!
# this cost is shipping + production costs
model.Cost()

30770.0

In [ ]:
# print the solution for how much goes in each node
for c in CUS:
    for s in SRC:
        print(c, s, model.x[c,s]())

P1 M1 0.0
P1 M2 0.0
P1 M3 35.0
P1 M4 75.0
P2 M1 0.0
P2 M2 0.0
P2 M3 115.0
P2 M4 0.0
P3 M1 95.0
P3 M2 0.0
P3 M3 0.0
P3 M4 40.0
P4 M1 30.0
P4 M2 100.0
P4 M3 0.0
P4 M4 0.0


# On Your Own

And recall that there is a linear relationship between the amount flowing each node and the amount. Just multiply the output above by the appropriate cost from the mine. Left to students as an exercise - good luck!

* Production Cost = 27,215
* Shipping Cost = 3,555
* Total Cost = 30,770

Also, remember from the beginning - you still have 5 tons of coal leftover in supply... where's the best place to send it if you could increase demand at any of the four power plants? This may require you to use a longer form of coding up the model (similar to Powell).